In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


![이미지 출력 필요](/content/drive/MyDrive/Study/Images/1.perceptron.png)

### 퍼셉트론의 학습

출력 = $F(w_0 + w_1 *x_1 + w_2 * x_2 + ... + w_n *x_n)$  = Weighted Sum  
$w_o$은 bias이다.
입력(x) * 가중치(w) + 편향(bias)   

#### 1.퍼셉트론 특징
1. 퍼셉트론은 hidden layer가 존재하지 않는다.
2. single layer로 구성되어있다.
 - input, weight, activation, output

#### 2.Activation function 이란?
1. 임계점(threshold)이 넘어가면 1, 안 넘어가면 0 이다.
2. 비선형으로 변형할 수 있다.



#### 3.How 예측값과 실제 값의 차이를 줄는지?
- 경사하강법(Gradient descent)을 사용
   



### 회귀(Regression) 개요
목표: 최적의 회귀 계수를 찾아낸다.  

$ Y = w_1 * x_1 + w_2 * x_2 + ... w_n * x_n$

1. x는 독립변수(feature), y는 종속변수
2. w는 회귀 계수(regression coefficients)

선형회귀는 완벽하게 정답에 대해 일직선 선을 그을 수 없다.  
실제 값과 예측 값의 차이(잔차)를 최소로 만드는 최적의 회귀 계수를 찾아야한다.  

### 오류 측정 방법
1. RSS(Residual Sum of Square)
    - 잔차 제곱의 합
    - 잔차를 다 더하면 0 이 나온다.
    - w 변수(회귀 계수)가 중심 변수임을 인지하는 것이 매우 중요한다.

2. MSE(Mean Squared Error)
    - RSS에서 학습 개수로 나눈 값이다.
    - MSE 는 비용(Cost)이며 w(회귀 계수)로 구성되는 MSE를 비용 함수라고 한다.
    - 머신 러닝 회귀 알고리즘은 데이터를 계속 학습하면서 비용 함수가 반환하는 값(오류값)을 계속해서 감소시키고 최종적으로 더 이상 감소하지 않는 최소의 오류 값을 구하는 것이다.
    - 비용 함수를 손실함수(loss fucntion)이라고 한다.

### 경사하강법
1. 어떻게 하면 오류가 작아지는 방향으로 W 값을 보정할 수 있을까? 
2. 미분은 증가 또는 감소의 방향성을 나타낸다.
3. Loss(w)를 편미분해 미분 함수의 최솟값을 구해야하는데, Loss(w)는 두 개의 파라미터인 $w_0$과 $w_1$을 각각 가지고 잇기 떄문에 편미분을 적용해야한다.
4. 가중치/절편 W 값은 손실 함수의 편미분 값을 Update 하면서 계속 갱신함.
5. Update는 기존 W 값에 손실 함수 편미분 값을 감소 시키는 방식을 정용하되 편미분 값을 그냥 감소 시키지 않고 일정한 계수를 곱해서 감소 시키며, 이를 학습률(learning rate)라고 한다.
6. 학습률이 큰 경우, 빠르게 학습이 가능 하지만 최소값을 지나칠 수 있다. 학습률이 작은 경우, 학습 속도가 오래 걸릴 수 있다.





In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
print(boston.keys()) 
# print(boston) # dictionary 타입으로 되어있다.

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [2]:
bostonDF = pd.DataFrame(boston.data, columns = boston.feature_names)
bostonDF['PRICE'] = boston.target     # 종속 변수 추가
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
# rm은 RM(방개수), lstat(하위계층 비율), target은 PRICE
# 반환 값은 weight와 bias가 update되어야 할 값과 Meas Squared Error 값을 loss로 반환.
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate):
    
    N = len(target)

    predicted = w1 * rm + w2  * lstat + bias   # 506개를 한 꺼번에 계산(column)
    diff = target - predicted     # 잔차(Residual)
    bias_factor = np.ones((N,))  # 편향(bias)

    w1_update = -(2/N) * learning_rate * (np.dot(rm.T, diff))      # w1를 기준으로 편미분
    w2_update = -(2/N) * learning_rate * (np.dot(lstat.T, diff))   # w1를 기준으로 편미분
    bias_update = -(2/N) * learning_rate * (np.dot(bias_factor.T, diff))

    mse_loss = np.mean(np.square(diff))

    return bias_update, w1_update, w2_update, mse_loss

In [4]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs 수만큼 반복적으로 Weight와 Bias를 update 적용.
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    w1 = np.zeros((1,))    # 원래는 랜덤 값으로 하는게 맞다.
    w2 = np.zeros((1,))
    bias = np.ones((1,))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 저장,호출 시 numpy array 형태로 RM과 LSTAT로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행
    for i in range(iter_epochs):
        # weight/bias update 값 계산
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print("Epoch: ", i + 1, "/", iter_epochs)
            print("w1:", w1, "w2:", w2, "bias:", bias, "loss", loss)

    return w1, w2, bias

## Gradient descent 적용
    - 정규화/표준화 작업을 미리 선행


In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])
print(scaled_features[:15])

[[0.57750527 0.08967991]
 [0.5479977  0.2044702 ]
 [0.6943859  0.06346578]
 [0.65855528 0.03338852]
 [0.68710481 0.09933775]
 [0.54972217 0.09602649]
 [0.4696302  0.29525386]
 [0.50028741 0.48068433]
 [0.39662771 0.7781457 ]
 [0.46809734 0.424117  ]
 [0.53956697 0.51655629]
 [0.46905537 0.31843267]
 [0.44606246 0.38576159]
 [0.45755892 0.18018764]
 [0.48572523 0.23537528]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

sclaer = MinMaxScaler()
scaled_feature = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

# scaled_feature 값을 넣는다.
# .value 는 array로 바꿔준다.
w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True )
print(" ### 최종 w1, w2, bias #####")
print(w1, w2, bias)

In [7]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(1, input_shape=(2,), activation = None, kernel_initializer='zeros', bias_initializer='ones')       
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit( x= scaled_features, y= bostonDF['PRICE'].values, epochs=1000)

In [9]:
predicted = model.predict(scaled_features)
bostonDF['predicted_price'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,predicted_price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.968979
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.495987
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.624313
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.402267
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.589417
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,28.097370
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.319248
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.744389
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.037947
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.241455
